# ⬇⬇⬇⬇⬇⬇⬇ Contrastive Learning for Entity-Matching ⬇⬇⬇⬇⬇⬇⬇

## Transformer Encoder For Enity matching
Using transformers to encode names can be very useful from various aspects 

In [1]:
!pip install -q -r requirements.txt

In [2]:
# !pip install -q -U sentence-transformers

# Import required dependencies 

In [24]:
import sys
import os
import pandas as pd
# import numpy as np
import numpy as np
import cupy as cp
from pathlib import Path
import sys 
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
sys.path.append(os.path.abspath('..'))

# Read data files

In [25]:
filtered_path = 'data/sp-company-data'
filtered_name = 'filtered_ciqcompany'
filtered_norm_path = 'data/sp-company-data-normalized'
filtered_norm_name = 'normalized'
col_names = ['companyid','companyname']


In [26]:
from matcher.utils import read_files

all_filtered_norm = read_files(filtered_norm_path, filtered_norm_name, col_names, sep='\t')
# all_filtered_source = read_files(filtered_path, filtered_name, col_names)

>> Num of files: 38


In [27]:
all_filtered_norm[all_filtered_norm.companyname.str.contains('1inch Ltd')]

,companyid,companyname
1040,684039648,1inch Ltd
1041,693531671,1inch Ltd


In [5]:
all_filtered_norm.shape

(10314918, 2)

In [21]:
# all_filtered_norm.drop_duplicates(['companyname'],inplace=True)

In [22]:
all_filtered_norm.set_index('companyname', inplace=True)

# Read Queries

In [8]:
# merged_data.to_csv('data/train/merged_norm_source.csv',index=False)

In [28]:
## Read matched frame
matched_frame  = pd.read_csv('data/inferess-sp-ciq-matched-names.tsv', sep='\t')
## read Infress database

inferess_file = ['data/preview-sec-gen-07-set-02-rev-01-2015-2022-company.tsv','2022-12-01-company.tsv' ][1]

inferess_comp = pd.read_csv(f'data/{inferess_file}', sep='\t')



In [10]:
inferess_comp.columns

Index(['valid_from', 'last_seen', 'inferess_entity_id', 'normalized_name',
       'reference_name', 'undisclosed', 'undisclosed_owner', 'cik', 'symbol',
       'isin', 'lei', 'ein', 'sic', 'state_of_incorporation', 'address'],
      dtype='object')

In [16]:
inferess_comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65176 entries, 0 to 65175
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   valid_from              65176 non-null  object 
 1   last_seen               65176 non-null  object 
 2   inferess_entity_id      65176 non-null  int64  
 3   normalized_name         65176 non-null  object 
 4   reference_name          22086 non-null  object 
 5   undisclosed             65176 non-null  bool   
 6   undisclosed_owner       6402 non-null   float64
 7   cik                     12206 non-null  float64
 8   symbol                  14159 non-null  object 
 9   isin                    11220 non-null  object 
 10  lei                     13261 non-null  object 
 11  ein                     11413 non-null  object 
 12  sic                     12041 non-null  float64
 13  state_of_incorporation  10837 non-null  object 
 14  address                 19724 non-null

In [15]:
inferess_comp.head(3)

,valid_from,last_seen,inferess_entity_id,normalized_name,reference_name,undisclosed,undisclosed_owner,cik,symbol,isin,lei,ein,sic,state_of_incorporation,address
0,2015-04-01,2017-03-24,70101953912,024 Pharma Inc,"024 Pharma, Inc.",False,NaN,1307969.0,NaN,NaN,NaN,20-1862731,3089.0,NJ,"224 DATURA STREET,WEST PALM BEACH,FL,33401"
1,2018-02-17,2019-02-14,90100082802,1 Customer - 0001159167,NaN,True,7.010160e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-02-27,2016-02-26,70101857119,1-800 Contacts Inc,1-800 Contacts Inc.,False,NaN,NaN,CTAC.XX,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Filter infress compaines by drop undisclosed
inferess_comp.drop(inferess_comp[inferess_comp.undisclosed == True].index, inplace=True)

In [30]:
inferess_comp = inferess_comp.reset_index(drop=True)

In [31]:
# Assert the undisclosed
assert all(inferess_comp.undisclosed.unique() == np.array([False]))

In [32]:
# Drop any duplicates

In [33]:
inferess_comp = inferess_comp.drop_duplicates(['normalized_name']).reset_index(drop=True)

In [34]:
inferess_comp.set_index('normalized_name', inplace=True)

In [35]:
import neattext as ntx
import re
def preprocess(x):
    x = re.sub(r"[\*\"\n\\…\+\-\/\=\(\)‘•€\[\]\|♫:;—,.”“~`#]", "", x) 

    # # x = ntx.replace_term(x, ' and ','')
    # # x = ntx.replace_term(x, '&','')
    # x = ntx.replace_term(x, 'aktiengesellschaft','ag')
    # x = ntx.replace_term(x, 'public company limited','pcl')
    # x = ntx.replace_term(x, 'public limited company','plc')
    # x = ntx.replace_term(x, 'limited partnership','lp')
    # x = ntx.replace_term(x, 'limited liability company','llc')    
    # x = ntx.replace_term(x, 'société anonyme','sa')
    # x = ntx.replace_term(x, 'the','')
    # x = ntx.replace_term(x, 'private','pvt')    
    # x = ntx.replace_term(x, 'limited','ltd')
    # x = ntx.replace_term(x, 'company','co')
    # x = ntx.replace_term(x, 'incorporated','inc')
    # x = ntx.replace_term(x, 'corporation','corp')
    # # x = ntx.replace_term(x, ' ','')

    return x 

In [118]:
a['norm_ws'] = a['norm'].apply(preprocess).apply(lambda x : ntx.replace_term(x, ' ','') ) 
b['norm_ws'] = b['norm'].apply(preprocess).apply(lambda x : ntx.replace_term(x, ' ','') ) 

In [62]:
a['f_word'] = a.norm.apply(lambda x : ''.join(x.split()[:-1]) if len(x.split()) > 1 else ''.join(x.split())).apply(lambda x : ntx.replace_term(x, ' ',''))
b['f_word'] = b.norm.apply(lambda x : ''.join(x.split()[:-1]) if len(x.split()) > 1 else ''.join(x.split())).apply(lambda x : ntx.replace_term(x, ' ',''))


In [25]:
import torch 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device 

'cuda'

# Model name

In [36]:

def predict_match(simcse_model, query,return_type='dict', top_k=3,threshold=0.1):
    """ Get the desired outpur format for Faiss searcher
    
    @param
    ------
    - simcse_model(SimCSE object): that alreay built an index matrix for the target data we want query
    - query(Union[List,str]): query list or str
    - return_type(str): defines the data shape or format you want to return based on needs 
    - top_k(int): Max number of matches to return for each record 
    - threshold(float): similarity score threshold
    
    @return
    -------
    dict or list of matches with scores
    """
    results =  simcse_model.search(query, top_k=top_k,threshold=threshold)

    if return_type == 'dict':
        return [{query[i]: results[i]} for i in range(len(query))]
    
    elif return_type=='predictions': 
        return [r[0][0] for r in results], [r[0][1] for r in results]
    elif return_type == 'matches':
        return results

In [37]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'

In [44]:
merged_data.sample(500000).reset_index(drop=True).to_csv('data/train/sample_merged.csv',index=False)

In [70]:
with open('titles.txt','r') as f:
    titles=[]
    for line in f:
        strip_lines=line.strip()[1:-1]
        m=titles.append(strip_lines)    

In [73]:
len(titles)

130

# Simcse Encoder Validation

In [38]:
from matcher.core import SimCSE_Matcher
model_path = 'sentence-transformers/all-MiniLM-L6-v2'
# model_name= 'data/artifacts/all-MiniLM-Norm/checkpoint-5000'
model_name = 'data/artifacts/all-MiniLM-Nli-v02'
# model_name= 'data/artifacts/all-MiniLM-Norm-v02'
# model_name = 'data/artifacts/all-MiniLM-Nli-All-Pos-v01'

# model_name = 'data/artifacts/all-MiniLM-Nli-All-Random-v02'
# model_name = 'data/artifacts/all-MiniLM-Nli-All-Real-v01'
model_name = 'data/artifacts/data/artifacts/all-MiniLM-Nli-All-Random-v03'


In [5]:

exp = 'all-MiniLM-Nli-All-Random-v4'
model_name = f'data/artifacts/{exp}'

In [39]:
simcse_model = SimCSE_Matcher(model_path)

# Big data searching


In [40]:
query_id = 'inferess_entity_id'
source_id = 'companyid'

In [41]:
all_filtered_norm.index

RangeIndex(start=0, stop=10314918, step=1)

In [23]:
%%time
matches = simcse_model.match_data(source_data= all_filtered_norm.index.tolist(),
                                    query_data= inferess_comp.index.tolist(),
                                    b_size=1000_000,
                                    top_k=1000,
                                    threshold=0.5)

01/03/2023 08:00:25 - INFO - matcher.core -   Encoding embeddings for sentences...
 58%|█████▊    | 9104/15625 [02:17<01:38, 66.28it/s]


KeyboardInterrupt: 

In [113]:
matches.to_json('data/All-Pos-V02-Matches.json')

In [115]:
pd.read_json('data/All-Pos-V02-Matches.json')

,query,pred,score,matches
0,024 Pharma Inc,024 Pharma Inc,1.000000,"[[024 Pharma Inc, 0.9999997020000001], [N4 Pharma PLC, 0.7421659827], [Naryx Pharma Inc, 0.7056563497], [XML4Pharma, 0.7020528316]]"
1,1-800 Contacts Inc,1-800 CONTACTS Inc,1.000000,"[[1-800 CONTACTS Inc, 1.0000001192], [1-800-Doctors Inc, 0.8588901162], [1-800 Communications Inc, 0.8151120543], [1 800 Heaters Inc, 0.8043341637], [1-800-PublicRelations Inc, 0.7630267739000001], [EveryDay Contacts Inc, 0.7625681162], [1-800-HOSTING Inc, 0.7427418232], [2M Creations Inc, 0.7395845652], [1-800-ATTORNEY Inc, 0.7262157798000001], [1 800 Collect Inc, 0.7245449424]]"
2,1-800-FLOWERS.COM Inc,1-800-FLOWERS.COM Inc,1.000000,"[[1-800-FLOWERS.COM Inc, 0.9999997616], [1-800-WINESHOP.COM Inc, 0.8846229911], [1-800-GIFTCARD.COM Inc, 0.8438376188000001], [Fiftyflowers.Com Inc, 0.8169343472], [1-800-Pharmacy Inc, 0.8128036261], [1-800-HealthPlan.com, 0.8112599254], [1-800 Communications Inc, 0.7981801033], [1-800 AutoTow Inc, 0.7858853936], [10monkeys.com Inc, 0.7813709378], [1.800.Vending Inc, 0.7813597918]]"
3,100 and RW CRA LLC,100 and RW Cra LLC,1.000000,"[[100 and RW Cra LLC, 1.0000001192], [300 and Colwell LLC, 0.7602412105], [100 Glen LLC, 0.7578610182000001], [1001 W. Exchange LLC, 0.7562142611], [64 and LWR LLC, 0.7536690235], [M and W Covington LLC, 0.7524853945000001], [FWB Bedford LLC, 0.7409630418], [2500 W. Maple LLC, 0.7408424020000001], [FTW Properties LLC, 0.7385591269], [W Power and Light LLC, 0.7376409769000001]]"
4,10Clouds,123Cloud,0.801979,"[[123Cloud, 0.8019786477], [Wanclouds Inc, 0.7991065383], [WhiteClouds Inc, 0.7798539996], [CoolClouds Inc, 0.7729571462], [NetCloud VPS, 0.7715476155000001], [Sharingcloud, 0.7649594545], [3Cloud LLC, 0.7634806037], [99cloud Inc, 0.7547730803], [Netcloud AG, 0.7378717661], [Netcloud Ltd, 0.7337325215]]"
...,...,...,...,...
58692,voxeljet AG,voxeljet AG,1.000000,"[[voxeljet AG, 0.9999998808], [Conrad-Storz AG, 0.8209168315], [Hotel Engel AG, 0.8182574511], [Lewisfield AG, 0.8136062622], [L Oberholzer AG, 0.8125008345], [Von Felbert AG, 0.8089028597], [Luco Container AG, 0.8029857874], [Live Billard AG, 0.8018601537000001], [LD Licht-Decor AG, 0.8005169034], [Paul Guatelli AG, 0.8004150987]]"
58693,withRovi Data Solutions,Witham Vehicle Solutions Ltd,0.784064,"[[Witham Vehicle Solutions Ltd, 0.7840636373000001], [Whitech Software Solutions, 0.7834324837000001], [Hulul Business Solutions, 0.7603263259], [Handd Business Solutions, 0.7573967576], [Method Media Solutions, 0.7489978671], [Focus Technology Solutions, 0.7479515672], [Windval Technology Solutions LLC, 0.7443363667], [Dog Media Solutions LLC, 0.7437873483], [Waupaca Sand and Solutions, 0.7422462702], [United Technology Solutions, 0.7405285239]]"
58694,www.robertgraham.Differential Brands Group Inc,Wolseley Industrial Products Group Inc,0.770486,"[[Wolseley Industrial Products Group Inc, 0.7704858184000001], [Wertimer Jones Marketing Group, The, 0.7670107484], [Wealth Builders Consulting Group Inc, 0.7532520294], [Washington Economics Consulting Group Inc, 0.7369043231], [Patsy Lou Williamson Automotive Group Inc, 0.7306575179], [Warren Township New Jersey School District, 0.7279858589], [Washington Township School Building Corp. (Porter County), 0.7249760628], [Wesco Aircraft Electronics Product Group, 0.7249084115000001], [Jacques Whitford Engineering Group Inc, 0.7245712876], [Shandong Ruyi Woolen Garment Group Co Ltd, 0.7218584418]]"
58695,x 500,Xelix,0.809567,"[[Xelix, 0.8095669746], [Xemax, 0.8062956929], [Xxl Co, 0.7963183522], [XOTOX, 0.7907164693000001], [X Fit, 0.7882877588], [XCO ehf, 0.7792802453000001], [X-girl, 0.7726336122], [XPLOR, 0.7718968391000001], [Xplor, 0.7718967795], [Xxl.ee, 0.7691780329]]"


In [121]:
threshold = 0.99
accepted = matches[matches['score']>=threshold]
not_accepted = matches[matches['score']< threshold]

In [122]:
cand_threshold = 0.85
candidates = not_accepted[not_accepted['score'] >= cand_threshold]
denied = not_accepted[not_accepted['score'] <cand_threshold]

In [123]:
accepted.shape, candidates.shape

((26192, 4), (14961, 4))

In [133]:
candidates['matches'] = candidates['matches'].apply(lambda x: x[:5])

/tmp/ipykernel_60/1483576840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['matches'] = candidates['matches'].apply(lambda x: x[:5])


In [141]:
def match_filter(x,threshold):
    filtered = [z if z[1] > threshold else None for z in x]
    return list(filter(None, filtered))

In [143]:
match_filter(candidates.iloc[12]['matches'], threshold=0.85)

[('4-Star Group Inc', 0.86519176), ('Fourth Group Inc', 0.85379064)]

In [145]:
candidates['matches'] = candidates['matches'].apply(lambda x : match_filter(x, 0.85))

/tmp/ipykernel_60/214364650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['matches'] = candidates['matches'].apply(lambda x : match_filter(x, 0.85))


In [107]:
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_row',100)

In [149]:
candidates[(candidates['score'] <threshold) &(candidates['score'] >=threshold-0.06) ].sort_values('score', ascending=True).head(100)

,query,pred,score,matches
1143,ARPI LLC,Arlo LLC,0.930004,"[(Arlo LLC, 0.9300041), (ARPAC LLC, 0.9172122), (Arut LLC, 0.9088817), (Argo LLC, 0.901259), (Ar Lc LLC, 0.9004116)]"
36593,New York Blood Center,New York Blood Center Inc,0.930009,"[(New York Blood Center Inc, 0.9300086)]"
38591,Opti Shell,Opti Build,0.930011,"[(Opti Build, 0.9300109), (Opti Soft, 0.9192379), (Opti Mise, 0.91380566), (Opti View, 0.90597326), (Opti-Ment, 0.8957242)]"
51230,Telephonics(r) Corp,Telephonics Corp,0.930022,"[(Telephonics Corp, 0.93002164)]"
35613,NVS,Jvs,0.930029,"[(Jvs, 0.93002945), (JVS, 0.9300293), (NV, 0.9202881), (NVM, 0.9123313), (NMV, 0.9103268)]"
41945,Professional Employer Services,Professional Employer Services Inc,0.930036,"[(Professional Employer Services Inc, 0.93003577), (Professional Employment Services Ltd, 0.9107562), (Professional Employment Services Inc, 0.89521277), (Professional Staffing Services, 0.87768424), (Professional Benefit Services, 0.87589747)]"
779,ALSP Orchid Acquisition Corp I,ALSP Orchid Acquisition Corporation I,0.930039,"[(ALSP Orchid Acquisition Corporation I, 0.9300394)]"
53572,US Customers,US Merchants,0.930055,"[(US Merchants, 0.9300555), (US Consults, 0.87674344), (Us-creations, 0.861402)]"
35119,NCAP,Ncpa,0.930066,"[(Ncpa, 0.9300662), (NCP, 0.9142167), (Ncip, 0.89232403), (Nca Inc, 0.8861174), (NCSA, 0.88546556)]"
17756,Energreen Corp Bhd,Energreen Corporation Bhd,0.930116,"[(Energreen Corporation Bhd, 0.93011606), (Envitech Sdn Bhd, 0.89040136), (ENRA ICON Sdn Bhd, 0.8897097), (Enlinea Sdn Bhd, 0.88786715), (ENRA IOL Sdn Bhd, 0.8866313)]"


In [115]:
x = 'Puretec inc '
y = 'Puretec ltd'
simcse_model.similarity(x, y)

100%|██████████| 1/1 [00:00<00:00, 164.12it/s]


0.9949986934661865

In [35]:
candidates.shape

NameError: name 'candidates' is not defined

In [116]:
candidates.sort_values('score',ascending=True).head(50)

,query,pred,score,matches
26789,Integrated Cabinet and Power Distribution,Integrated Data Processing and Payment Systems Inc,0.850012,"[(Integrated Data Processing and Payment Systems Inc, 0.85001194), (Integrated Computer Sales And Service Inc, 0.8468217), (Integrated Finance and Accounting Solutions LLC, 0.84603065), (Integrated Distribution and Logistics Direct LLC, 0.8430939), (Integrated Controls and Information LLC, 0.841277), (Integrated Insurance and Risk Management Inc, 0.83793646), (Integrated Systems And Power Inc, 0.8333139), (Integrated Energy Distribution and Marketing Ltd, 0.82897425), (Integrated Commercial and Industrial Design Ltd, 0.82669413), (Integrated Computer Solutions Of Vestal LLC, 0.8247819)]"
54420,UnitedHealthcare Healthy Rewards,UnitedHealthcare Specialty Benefits LLC,0.850037,"[(UnitedHealthcare Specialty Benefits LLC, 0.85003656), (UnitedHealthcare StudentResources, 0.82027996), (UnitedHealthcare Benefits Plan of California Inc, 0.799724), (UnitedHealthcare Accelerator Powered by Techstars, 0.7990475), (UnitedHealthcare Integrated Services Inc, 0.79757845), (UnitedHealthcare Benefits of Texas Inc, 0.7899035), (UnitedHealthcare Insurance Co of the River Valley, 0.7874952), (UnitedHealthcare National Accounts, 0.7846725), (UnitedHealthcare Insurance Designated Activity Co, 0.7815659), (UnitedHealthcare Services Co of the River Valley Inc, 0.77773136)]"
52560,Transition Services,TUI Destination Services,0.850038,"[(TUI Destination Services, 0.850038), (T.P. Technical Services, 0.844251), (Transact ATM Services, 0.8370986), (Technologies Et Services, 0.8370355), (Techtron Services, 0.8318261), (Tm Transport Services, 0.83019584), (Transition Services Ltd, 0.82301223), (T and T Cleaning Services, 0.812789), (TRA Professional Services, 0.8111885), (Tecom-Vinnell Services, 0.8013582)]"
20584,Fred Hutchison Research Center,Fred Hutchinson Cancer Center,0.850044,"[(Fred Hutchinson Cancer Center, 0.85004437), (Woods Hole Research Center, 0.8263776), (Fred Hutchinson Cancer Research Center, Education Centre, 0.8123143), (Wockhardt Research Center, 0.8025463), (Helen Hamlyn Research Centre, 0.8022626), (Wes Watkins Technology Center, 0.79928607), (Wockhardt Research Centre, 0.7960092), (The Wakasa Wan Energy Research Center, 0.7891271), (BD Garware Research Centre, 0.7837941), (Robert W. Franz Cancer Research Center, 0.7797352)]"
28347,KARD Scientific,Kerona Scientific,0.850056,"[(Kerona Scientific, 0.85005563), (KARD Scientific Inc, 0.84445894), (Nawah Scientific, 0.80012524), (K Property SCI, 0.78134996), (Kannar Earth Science, 0.780121), (Kao Yuan University, 0.7654365), (Klar Scientific Inc, 0.7624342), (Kabarak University, 0.7605438), (kj scientific, 0.7604785), (kaputt.de GmbH, 0.7602162)]"
55534,Veterans Administration and Department of Defense,Veterans Contracting and Procurement LLC,0.850060,"[(Veterans Contracting and Procurement LLC, 0.8500603), (Veteran Development And Construction LLC, 0.8097681), (Veterans Plaza Authority,Iowa, 0.7905249), (Veterans First Initiative LLC, 0.7871076), (Veterans Administration Federal Credit Union, 0.7837518), (Veteran Sales And Service LLC, 0.7832839), (Veteran Engineering and Technology LLC, 0.7789064), (Veterans Of Foreign Wars Of The United States, 0.7783232), (West Virginia Department of Finance and Administration, 0.77778816), (Veterans Health Administration Credit Union, 0.7768815)]"
24075,Health Canada Regulatory,Health Products Regulatory Authority,0.850061,"[(Health Products Regulatory Authority, 0.85006094), (Health Robotics Canada LLC, 0.83250326), (Health Council of Canada, 0.823419), (Health Canada, 0.811173), (Health Action Partnership International C.I.C, 0.8059207), (Health Risk Interactive LLC, 0.7814057), (Healthpro Federal Credit Union, 0.7785003), (Healthcare Insurance Reciprocal of Canada, 0.7778525), (Health and Safety Compliance Partnership Ltd, 0.7711435), (Health Care Idaho Credit Union, 0.77056164)]"
1407,AURA Networks,Aura Network,0.85

In [117]:
accepted.sort_values('score',ascending=True).head(100)

,query,pred,score,matches
53061,Twentieth Century Fox Licensing and Merchandising,Twentieth Century Fox Licensing and Merchandising Ltd,0.980027,"[(Twentieth Century Fox Licensing and Merchandising Ltd, 0.9800275), (Twentieth Century Fox Licensing And Merchandising Corp, 0.9623263), (Twentieth Century Fox Film Corporation, Betriebsgesellschaft Fur Die Schweiz, 0.8678591), (Twentieth Century Fox Home Entertainment Germany GmbH, 0.81811094), (Twentieth Century Fox Home Entertainment Italia S.r.l, 0.8036167), (Twentieth Century Fox Home Entertainment LLC, 0.8003604), (Twentieth Century Fox Film Company (Export) Ltd, 0.79207236), (Twentieth Century Fox Home Entertainment Ltd, 0.7884891), (Twentieth Century Fox of Germany GmbH, 0.7714971), (Twentieth Century Fox Home Entertainment Inc, 0.76085234)]"
38714,Orchid Apartments Pvt Ltd,Orchids Apartments Pvt Ltd,0.980035,"[(Orchids Apartments Pvt Ltd, 0.98003495), (Orchid Recruiter Pvt Ltd, 0.915595), (Orchid Overseas Pvt Ltd, 0.9132987), (Orchid Solution Pvt Ltd, 0.9119587), (Orchid Laminates Pvt Ltd, 0.90672886), (Orchid Shipping Pvt Ltd, 0.90532684), (Orchid Hotels Pune Pvt Ltd, 0.9011218), (Orchid Infrastructure Developers Pvt Ltd, 0.89375687), (Aditya Builders Pvt Ltd, 0.88971657), (Orchid Industries Pvt Ltd, 0.888806)]"
25614,IM Es,Imes,0.980099,"[(Imes, 0.98009896), (Imtes, 0.9630199), (IMS, 0.949726), (Ims, 0.949726), (Imrs, 0.93247384), (Imbs, 0.9247764), (IMS AS, 0.9180928), (Imsas, 0.910771), (Imo As, 0.9042795), (Innes, 0.8826153)]"
20732,"Fruci and Associates, II PLLC",Fruci and Associates II PLLC,0.980134,"[(Fruci and Associates II PLLC, 0.9801336), (Frisco Eye Associates PLLC, 0.83931184), (Frasier, Dean and Howard PLLC, 0.74978733), (FC Atlantic Commons Phase II LLC, 0.7469597), (FRO II Montrouge Holdings II LLC, 0.74625456), (Creason and Associates PLLC, 0.7448987), (CF Industries Canada Investment ULC, 0.7411377), (Taylor and Associates PLLC, 0.7394643), (Costanzo and Associates PLLC, 0.73408854), (Fucci Law and Adr PLLC, 0.7317553)]"
18246,Essex Property Trust Inc,Essex Properties Trust Inc,0.980143,"[(Essex Properties Trust Inc, 0.9801428), (Essex Youth Trust, 0.8771386), (Oxford Property Trust, 0.87027943), (Arbor Realty Trust Inc, 0.8534901), (The Essex Investment Trust, 0.8495348), (The Essex Heritage Trust, 0.8467391), (The Essex Road Trust, 0.84173036), (Essex Gardens Trust, 0.8384508), (Owens Family Trust, 0.8361189), (ALE Direct Property Trust, 0.83296764)]"
12018,Coca-Cola Co,The Coca-Cola Co,0.980144,"[(The Coca-Cola Co, 0.9801441), (Coken Co Inc, 0.8760983), (The Fitch Co, 0.8602354), (Coa F and T Co, 0.8516917), (The Coral Co, 0.83063877), (Coca-Cola AG, 0.8304198), (C and B Silo Co, 0.8286418), (The Cafaro Co, 0.8269315), (Canopy Co, 0.82352567), (CA Canada Co, 0.82159764)]"
4575,Association for the Advancement of Medical Instrumentation,Association For The Advancement Of Medical Instrumentation Inc,0.980176,"[(Association For The Advancement Of Medical Instrumentation Inc, 0.98017615), (Association for Consultancy and Engineering, 0.89305764), (Association For Search and Technical Rescue, 0.88728505), (Association For Career And Technical Education Inc, 0.88533294), (Association For Cognitive Analytic Therapy, 0.8839964), (Association For Clinical Theological Training And Care Limited(The), 0.87991774), (Association For Supervision And Curriculum Development, 0.87964773), (Association for Surgical Education, 0.87959445), (Association for the Advancement of Artificial Intelligence Inc, 0.87848324), (Association For Relief And Medical Aid, 0.87784094)]"
14622,Dahua Technology Co Ltd,Dahan Technology Co Ltd,0.980181,"[(Dahan Technology Co Ltd, 0.98018056), (Dawan Technology Co Ltd, 0.9767775), (Daa Sheen Technology Co Ltd, 0.94908315), (Dae Jin Technology Co Ltd, 0.9484427), (Dake Data Technology Co Ltd, 0.9352836), (Dae Gun Technology Co Ltd, 0.9281781), (Daeho Technology Co Ltd, 0.92165715), (Dawon HighTech Co Ltd, 0.9153648), (Dae Yang Tech Co Ltd, 0.912

In [156]:
accepted['query_id'] = inferess_comp.loc[accepted['query']][query_id].tolist()
accepted['source_id'] = all_filtered_norm.loc[accepted['pred']][source_id].tolist()

/tmp/ipykernel_60/175366808.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['query_id'] = inferess_comp.loc[accepted['query']][query_id].tolist()
/tmp/ipykernel_60/175366808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['source_id'] = all_filtered_norm.loc[accepted['pred']][source_id].tolist()


In [157]:
accepted.drop(['matches'], axis = 1, inplace = True) 

/tmp/ipykernel_60/3403730227.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted.drop(['matches'], axis = 1, inplace = True)


In [158]:
accepted

,query,pred,score,query_id,source_id
0,024 Pharma Inc,024 Pharma Inc,1.0,70101953912,51788819
1,1-800 Contacts Inc,1-800 CONTACTS Inc,1.0,70101857119,378329
2,1-800-FLOWERS.COM Inc,1-800-FLOWERS.COM Inc,1.0,70101859206,24085
3,100 and RW CRA LLC,100 and RW Cra LLC,1.0,70101659003,237393026
5,10X Capital Venture Acquisition Corp,10X Capital Venture Acquisition Corp,1.0,70101855520,689627552
...,...,...,...,...,...
58686,uniQure NV,uniQure NV,1.0,70101910357,171029664
58688,urban-gro Inc,urban-gro Inc,1.0,70101854751,431844712
58690,vTv Therapeutics Inc,vTv Therapeutics Inc,1.0,70101576553,304063261
58691,vitacost.com Inc,Vitacost.com Inc,1.0,70101938898,1057357


In [159]:
from tqdm.auto import tqdm 

candidate_dict = {'query_id':[], 'candidates_ids':[]}
for i,row in tqdm(candidates.iterrows()): 
    
    q_id= inferess_comp.loc[row['query']][query_id]
    
    candidate_dict['query_id'].append(q_id)
    c_names = [c[0] for c in row['matches']][:10]
    c_ids = all_filtered_norm.loc[c_names][source_id].tolist()
    
    candidate_dict['candidates_ids'].append(','.join([str(cid) for cid in c_ids])) 


0it [00:00, ?it/s]

In [160]:
import json 
cand = pd.DataFrame(candidate_dict)


,query_id,candidates_ids
0,90101591655,"595097235,650003033,39506238,589155560,1676271688"
1,90100030466,1197688
2,70101953888,"290625296,30946332"
3,90100545928,"47318725,46611940,3147488,27989161,336322257"
4,90100658623,539104735
...,...,...
14956,90101419514,"652626240,660322363,643947755,33281063,416386"
14957,90100761312,"417155,421047023,50984575"
14958,90100649759,24926975
14959,70101915758,"100887,98584"


In [166]:
x = cand.iloc[0]['candidates_ids']
x.split(',')

['595097235', '650003033', '39506238', '589155560', '1676271688']

In [167]:
cand.to_csv('data/output/candidates_v02.tsv', sep='\t',index =False)

In [169]:
accepted.to_csv('data/output/accepted_v02.tsv', sep='\t',index =False)